# Exploratory Data Analysis 


## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load the data
df = pd.read_csv('../data_data/AmesHousing.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## 2. Basic Data Inspection

### Data Types and Info

In [ ]:
# Data types inspection
print("Data Types:")
print(df.dtypes.value_counts())
print(f"\nTotal columns: {df.shape[1]}")
print(f"Total rows: {df.shape[0]}")

In [ ]:
# Summary statistics for numerical columns
print("Summary Statistics for Numerical Columns:")
df.describe()

In [ ]:
# Info about the dataset
df.info()

### Key Insights from Basic Inspection

**Data Types:**
- 11 float64 columns (expected numerical)
- 28 int64 columns (expected numerical)
- 43 object columns (expected categorical)
- Total: 2930 entries and 82 columns

**Target Variable (SalePrice):**
- Mean: $180,796
- Std: $79,886
- Range: Varies significantly

**Important Numerical Features:**
- `Overall Qual`: Quality ranges from 1-10 (mean ~6.09)
- `Year Built`: From 1872 to 2010
- `Lot Area`: Ranges widely with potential outliers
- `Gr Liv Area`: Above ground living area (mean ~1499 sq ft)

## 3. Missing Values Analysis

In [ ]:
# Calculate missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})

# Sort by missing percentage
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print("Columns with Missing Values:")
print(missing_data)

In [ ]:
# Visualize missing values
plt.figure(figsize=(14, 8))
missing_data_sorted = missing_data.sort_values('Missing_Percentage')
plt.barh(missing_data_sorted['Column'], missing_data_sorted['Missing_Percentage'])
plt.xlabel('Missing Percentage (%)')
plt.title('Missing Values by Column')
plt.tight_layout()
plt.show()

In [ ]:
# Missing values heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.tight_layout()
plt.show()

### Missing Values Insights

**High Missing Values (>50%):**
- `Alley`: 93.4% missing - almost unusable
- `Pool QC`: 99.5% missing - very few houses have pools
- `Misc Feature`: 96.4% missing
- `Fence`: 80.5% missing
- `Mas Vnr Type`: 60.6% missing

**Moderate Missing Values (~5%):**
- Garage-related features: Likely houses without garages
- Basement-related features: Likely houses without basements

**Low Missing Values (<3%):**
- Can be imputed with mean/median/mode

**Consideration:**
- Might set a threshold for acceptable % missing value to drop these columns, which seems to have both random and systematic data issue.

## 4. Outlier Analysis

In [ ]:
# Select key numerical features for outlier detection
numerical_features = ['SalePrice', 'Gr Liv Area', 'Lot Area', 'Total Bsmt SF', 'Overall Qual', 'Year Built']

# Create box plots for outlier visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, col in enumerate(numerical_features):
    axes[idx].boxplot(df[col].dropna(), vert=True)
    axes[idx].set_title(f'Box Plot: {col}')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary of potential outliers
print("Outlier Detection Summary (Using IQR Method):")
print("=" * 80)
for col in numerical_features:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(df)) * 100
    
    print(f"\n{col}:")
    print(f"  Lower Bound: {lower_bound:.2f}")
    print(f"  Upper Bound: {upper_bound:.2f}")
    print(f"  Outliers Count: {outlier_count} ({outlier_percentage:.2f}%)")
    if outlier_count > 0:
        print(f"  Min Outlier: {outliers.min():.2f}")
        print(f"  Max Outlier: {outliers.max():.2f}")

In [ ]:
# Detailed outlier analysis for SalePrice and Gr Liv Area
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# SalePrice - Histogram with outlier threshold
Q1_price = df['SalePrice'].quantile(0.25)
Q3_price = df['SalePrice'].quantile(0.75)
IQR_price = Q3_price - Q1_price
upper_price = Q3_price + 1.5 * IQR_price

axes[0, 0].hist(df['SalePrice'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(upper_price, color='red', linestyle='--', linewidth=2, label=f'Upper Threshold: ${upper_price:,.0f}')
axes[0, 0].set_xlabel('Sale Price')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('SalePrice Distribution with Outlier Threshold')
axes[0, 0].legend()

# SalePrice - Box plot
axes[0, 1].boxplot(df['SalePrice'], vert=True)
axes[0, 1].set_ylabel('Sale Price')
axes[0, 1].set_title('SalePrice Box Plot')
axes[0, 1].grid(True, alpha=0.3)

# Gr Liv Area - Scatter with outliers highlighted
Q1_area = df['Gr Liv Area'].quantile(0.25)
Q3_area = df['Gr Liv Area'].quantile(0.75)
IQR_area = Q3_area - Q1_area
upper_area = Q3_area + 1.5 * IQR_area

normal_data = df[df['Gr Liv Area'] <= upper_area]
outlier_data = df[df['Gr Liv Area'] > upper_area]

axes[1, 0].scatter(normal_data['Gr Liv Area'], normal_data['SalePrice'], alpha=0.5, label='Normal', s=30)
axes[1, 0].scatter(outlier_data['Gr Liv Area'], outlier_data['SalePrice'], color='red', alpha=0.7, label='Outliers', s=50)
axes[1, 0].axvline(upper_area, color='red', linestyle='--', linewidth=2, label=f'Threshold: {upper_area:.0f}')
axes[1, 0].set_xlabel('Above Ground Living Area (sq ft)')
axes[1, 0].set_ylabel('Sale Price')
axes[1, 0].set_title('Gr Liv Area vs SalePrice (Outliers Highlighted)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Lot Area - Box plot
axes[1, 1].boxplot(df['Lot Area'].dropna(), vert=True)
axes[1, 1].set_ylabel('Lot Area (sq ft)')
axes[1, 1].set_title('Lot Area Box Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identify extreme outliers in key features
print("\nExtreme Outliers Analysis:")
print("=" * 80)

# SalePrice outliers
price_outliers = df[df['SalePrice'] > upper_price][['SalePrice', 'Gr Liv Area', 'Overall Qual', 'Neighborhood']]
print(f"\nHigh-Priced Outliers (SalePrice > ${upper_price:,.0f}):")
print(f"Count: {len(price_outliers)}")
if len(price_outliers) > 0:
    print(price_outliers.sort_values('SalePrice', ascending=False).head(10))

# Gr Liv Area outliers
area_outliers = df[df['Gr Liv Area'] > upper_area][['Gr Liv Area', 'SalePrice', 'Overall Qual', 'Neighborhood']]
print(f"\n\nLarge Living Area Outliers (Gr Liv Area > {upper_area:.0f} sq ft):")
print(f"Count: {len(area_outliers)}")
if len(area_outliers) > 0:
    print(area_outliers.sort_values('Gr Liv Area', ascending=False).head(10))

# Unusual cases: Large area but low price
unusual = df[(df['Gr Liv Area'] > 4000) & (df['SalePrice'] < 200000)][['Gr Liv Area', 'SalePrice', 'Overall Qual', 'Neighborhood']]
print(f"\n\nUnusual Cases (Large Area but Low Price):")
print(f"Count: {len(unusual)}")
if len(unusual) > 0:
    print(unusual)

### Outlier Detection Insights

**Key Findings:**

**SalePrice Outliers:**
- High-end luxury homes exist above the upper threshold
- These represent legitimate expensive properties, not data errors
- Most outliers are in premium neighborhoods
- Consider keeping these for model training as they represent real market segments

**Gr Liv Area (Living Area) Outliers:**
- Several homes with exceptionally large living areas (>4,000 sq ft)
- Some large homes have surprisingly low prices → potential data quality issues or special circumstances
- These unusual cases should be investigated further
- May indicate foreclosures, incomplete properties, or data entry errors

**Lot Area Outliers:**
- Significant outliers with extremely large lot sizes
- Common in certain neighborhoods (likely rural or estate properties)
- Wide variance suggests different property types in the dataset

**Total Bsmt SF Outliers:**
- Some homes with unusually large basement areas
- Generally consistent with overall property size

**Recommendations for Modeling:**
1. **Keep most outliers**: They represent legitimate variation in housing market
2. **Investigate unusual cases**: Large area + low price combinations may be errors
3. **Consider robust models**: Use models resistant to outliers (e.g., tree-based methods)
4. **Winsorization option**: Cap extreme values at 95th or 99th percentile if needed
5. **Log transformation**: Apply to SalePrice and area features to reduce outlier impact
6. **Separate modeling**: Consider separate models for different price segments

**Outliers to Watch:**
- Homes with Gr Liv Area > 4,000 sq ft but SalePrice < $200,000
- Properties with Lot Area > 100,000 sq ft (potential acreage/farms)
- Extremely low or high Overall Quality scores combined with contradictory prices

## 5. Categorical DeepDive

#### Detailed Analysis: Roof Style

In [ ]:
# Roof Style analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Distribution - Pie chart
roof_counts = df['Roof Style'].value_counts()
axes[0, 0].pie(roof_counts.values, labels=roof_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Roof Style Distribution')

# Count bar chart
axes[0, 1].bar(range(len(roof_counts)), roof_counts.values, color='brown', alpha=0.7)
axes[0, 1].set_xticks(range(len(roof_counts)))
axes[0, 1].set_xticklabels(roof_counts.index, rotation=45, ha='right')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Roof Style Frequency')
axes[0, 1].grid(axis='y', alpha=0.3)

# Add counts on bars
for i, (roof, count) in enumerate(roof_counts.items()):
    axes[0, 1].text(i, count, str(count), ha='center', va='bottom')

# Price by roof style
df.boxplot(column='SalePrice', by='Roof Style', ax=axes[1, 0], rot=45)
axes[1, 0].set_xlabel('Roof Style')
axes[1, 0].set_ylabel('Sale Price')
axes[1, 0].set_title('Sale Price by Roof Style')

# Mean price comparison
roof_stats = df.groupby('Roof Style')['SalePrice'].agg(['mean', 'median', 'count', 'std']).round(0)
roof_stats = roof_stats.sort_values('mean', ascending=False)

axes[1, 1].barh(range(len(roof_stats)), roof_stats['mean'].values, color='sienna', alpha=0.7)
axes[1, 1].set_yticks(range(len(roof_stats)))
axes[1, 1].set_yticklabels(roof_stats.index)
axes[1, 1].set_xlabel('Mean Sale Price ($)')
axes[1, 1].set_title('Average Price by Roof Style')
axes[1, 1].invert_yaxis()

# Add count labels
for i, (idx, row) in enumerate(roof_stats.iterrows()):
    axes[1, 1].text(row['mean'], i, f"  n={int(row['count'])}", va='center')

plt.tight_layout()
plt.show()

print("Roof Style Statistics:")
print("=" * 80)
print(roof_stats.to_string())

# Identify dominant vs rare
print("\n" + "=" * 80)
print("CATEGORY CLASSIFICATION:")
for roof, count in roof_counts.items():
    pct = (count / len(df)) * 100
    category = "DOMINANT" if pct > 30 else ("MODERATE" if pct > 5 else "RARE")
    print(f"  {roof}: {count} ({pct:.2f}%) - {category}")

**Roof Style Insights:**
- **Extreme dominance:** Gable roofs represent ~86% of all houses - overwhelming majority
- **Secondary type:** Hip roofs ~18% - only other meaningful category
- **Very rare styles:** Gambrel, Mansard, Shed each <1% - extreme rarity (single-digit observations)
- **Price variation:** Hip roofs show slightly higher mean prices than Gable
- **Rare category problem:** Flat, Gambrel, Mansard, Shed too rare for reliable statistical inference
- **Overfitting risk:** Models may memorize rare roof styles instead of learning meaningful patterns
- **Recommended preprocessing:** 
  - Create binary feature: `Is_Gable` vs `Other`
  - Or group as: `Gable`, `Hip`, `Other` (combining rare types)
- **Encoding strategy:** After grouping, use one-hot encoding or keep as ordinal

#### Detailed Analysis: Foundation Type

In [ ]:
# Foundation Type analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distribution
foundation_counts = df['Foundation'].value_counts().sort_values(ascending=False)
colors = plt.cm.Set3(range(len(foundation_counts)))
axes[0].bar(range(len(foundation_counts)), foundation_counts.values, color=colors)
axes[0].set_xticks(range(len(foundation_counts)))
axes[0].set_xticklabels(foundation_counts.index, rotation=45, ha='right')
axes[0].set_ylabel('Count')
axes[0].set_title('Foundation Type Distribution')
axes[0].grid(axis='y', alpha=0.3)

# Add percentage and count labels
for i, (found, count) in enumerate(foundation_counts.items()):
    pct = (count / len(df)) * 100
    axes[0].text(i, count, f'{count}\n({pct:.1f}%)', ha='center', va='bottom', fontsize=9)

# Price distribution by foundation
df.boxplot(column='SalePrice', by='Foundation', ax=axes[1], rot=45)
axes[1].set_xlabel('Foundation Type')
axes[1].set_ylabel('Sale Price')
axes[1].set_title('Sale Price Distribution by Foundation Type')

# Mean price and count
foundation_stats = df.groupby('Foundation').agg({
    'SalePrice': ['mean', 'count']
}).round(0)
foundation_stats.columns = ['Mean_Price', 'Count']
foundation_stats = foundation_stats.sort_values('Mean_Price', ascending=False)

axes[2].barh(range(len(foundation_stats)), foundation_stats['Mean_Price'].values, 
             color=colors[:len(foundation_stats)], alpha=0.8)
axes[2].set_yticks(range(len(foundation_stats)))
axes[2].set_yticklabels(foundation_stats.index)
axes[2].set_xlabel('Mean Sale Price ($)')
axes[2].set_title('Average Price by Foundation Type')
axes[2].invert_yaxis()

# Add count labels
for i, (idx, row) in enumerate(foundation_stats.iterrows()):
    axes[2].text(row['Mean_Price'], i, f"  n={int(row['Count'])}", va='center')

plt.tight_layout()
plt.show()

print("Foundation Type Analysis:")
print("=" * 70)
foundation_summary = df.groupby('Foundation')['SalePrice'].describe()[['count', 'mean', 'std', 'min', 'max']]
print(foundation_summary.round(0).to_string())

# Identify rare categories
print("\n" + "=" * 70)
print("RARE CATEGORY ALERT:")
for found, count in foundation_counts.items():
    pct = (count / len(df)) * 100
    if pct < 5:
        print(f"  {found}: {count} ({pct:.2f}%) - RARE CATEGORY")

**Foundation Type Insights:**
**Distribution & Price Patterns:**
- **PConc (Poured Concrete):** ~50% of homes, highest avg price (~ 190,000) - modern, preferred
- **CBlock (Cinder Block):** ~37%, moderate pricing (~ 160,000) - traditional construction
- **BrkTil (Brick & Tile):** ~10%, moderate prices - older homes
- **Rare types (<2% each):** Slab, Stone, Wood - very few observations, lowest prices

**Recommended Preprocessing:**

1. **Group rare categories:** Combine `Slab`, `Stone`, `Wood` into `"Other_Foundation"`
2. **Final categories:** `PConc`, `CBlock`, `BrkTil`, `Other_Foundation` (4 total)
3. **Encoding strategy:** One-hot encoding (manageable with 4 categories)
4. **Optional feature engineering:** Create binary `Is_Modern_Foundation` (PConc=1, Others=0)
**Modeling Note:** Foundation type correlates with both home age and price; safe predictor after grouping rare categories.


#### Detailed Analysis: Garage Type

In [ ]:
# Garage Type analysis (including missing values)
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Create a copy with 'No Garage' for missing values
df_garage = df.copy()
df_garage['Garage Type'] = df_garage['Garage Type'].fillna('No Garage')

# Distribution
garage_counts = df_garage['Garage Type'].value_counts().sort_values(ascending=False)
axes[0, 0].bar(range(len(garage_counts)), garage_counts.values, color='darkgreen', alpha=0.7)
axes[0, 0].set_xticks(range(len(garage_counts)))
axes[0, 0].set_xticklabels(garage_counts.index, rotation=45, ha='right')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Garage Type Distribution (including No Garage)')
axes[0, 0].grid(axis='y', alpha=0.3)

# Add percentage labels
for i, (garage, count) in enumerate(garage_counts.items()):
    pct = (count / len(df)) * 100
    axes[0, 0].text(i, count, f'{pct:.1f}%', ha='center', va='bottom')

# Price by garage type
sns.violinplot(data=df_garage, x='Garage Type', y='SalePrice', ax=axes[0, 1])
axes[0, 1].set_xlabel('Garage Type')
axes[0, 1].set_ylabel('Sale Price')
axes[0, 1].set_title('Price Distribution by Garage Type')
axes[0, 1].tick_params(axis='x', rotation=45)

# Mean price comparison
garage_stats = df_garage.groupby('Garage Type').agg({
    'SalePrice': ['mean', 'median', 'count', 'std']
}).round(0)
garage_stats.columns = ['Mean', 'Median', 'Count', 'Std']
garage_stats = garage_stats.sort_values('Mean', ascending=False)

axes[1, 0].barh(range(len(garage_stats)), garage_stats['Mean'].values, 
                color=['red' if idx=='No Garage' else 'green' for idx in garage_stats.index],
                alpha=0.7)
axes[1, 0].set_yticks(range(len(garage_stats)))
axes[1, 0].set_yticklabels(garage_stats.index)
axes[1, 0].set_xlabel('Mean Sale Price ($)')
axes[1, 0].set_title('Average Price by Garage Type')
axes[1, 0].invert_yaxis()

# Add count and std labels
for i, (idx, row) in enumerate(garage_stats.iterrows()):
    axes[1, 0].text(row['Mean'], i, f"  n={int(row['Count'])}", va='center')

# Statistical comparison
axes[1, 1].axis('off')
table_data = []
for idx, row in garage_stats.iterrows():
    table_data.append([idx, int(row['Count']), f"${row['Mean']:,.0f}", 
                       f"${row['Median']:,.0f}", f"${row['Std']:,.0f}"])

table = axes[1, 1].table(cellText=table_data, 
                         colLabels=['Garage Type', 'Count', 'Mean Price', 'Median Price', 'Std Dev'],
                         cellLoc='center', loc='center',
                         colWidths=[0.2, 0.15, 0.2, 0.2, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
axes[1, 1].set_title('Garage Type Statistics Summary')

plt.tight_layout()
plt.show()

# Analyze impact of garage presence
has_garage = df['Garage Type'].notna()
print("\nGarage Presence Impact:")
print(f"Houses with garage: {has_garage.sum()} ({has_garage.sum()/len(df)*100:.1f}%)")
print(f"Houses without garage: {(~has_garage).sum()} ({(~has_garage).sum()/len(df)*100:.1f}%)")
print(f"\nMean price with garage: ${df[has_garage]['SalePrice'].mean():,.2f}")
print(f"Mean price without garage: ${df[~has_garage]['SalePrice'].mean():,.2f}")
print(f"Price difference: ${df[has_garage]['SalePrice'].mean() - df[~has_garage]['SalePrice'].mean():,.2f}")
print(f"Percentage increase: {((df[has_garage]['SalePrice'].mean() / df[~has_garage]['SalePrice'].mean()) - 1) * 100:.1f}%")

**Garage Type Insights:**
- **Dominant type:** Attached garages (~60%) most common, followed by Detached (~25%)
- **Missing values:** ~5% houses have no garage data - likely houses without garages (informative missingness)
- **Price impact:** Garage presence significantly increases home value (~$50k+ premium)
- **Type hierarchy:** Built-in garages command highest prices, CarPort lowest among garage types
- **No garage penalty:** Houses without garages sell for substantially less
- **Feature engineering opportunity:** Create binary `Has_Garage` feature (presence/absence indicator)
- **Encoding strategy:** One-hot encoding + missing indicator, or target encoding

#### Detailed Analysis: House Style

In [ ]:
# House Style analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distribution
style_counts = df['House Style'].value_counts().sort_values(ascending=True)
axes[0].barh(range(len(style_counts)), style_counts.values, color='mediumpurple')
axes[0].set_yticks(range(len(style_counts)))
axes[0].set_yticklabels(style_counts.index)
axes[0].set_xlabel('Count')
axes[0].set_title('House Style Distribution')
axes[0].grid(axis='x', alpha=0.3)

# Add percentage labels
for i, v in enumerate(style_counts.values):
    pct = (v / len(df)) * 100
    axes[0].text(v, i, f' {pct:.1f}%', va='center')

# Price by house style
df.boxplot(column='SalePrice', by='House Style', ax=axes[1], rot=45)
axes[1].set_xlabel('House Style')
axes[1].set_ylabel('Sale Price')
axes[1].set_title('Sale Price by House Style')

# Mean price and count
style_stats = df.groupby('House Style').agg({
    'SalePrice': ['mean', 'median', 'count']
}).round(0)
style_stats.columns = ['Mean', 'Median', 'Count']
style_stats = style_stats.sort_values('Mean', ascending=False)

axes[2].barh(range(len(style_stats)), style_stats['Mean'].values, color='orange', alpha=0.7)
axes[2].set_yticks(range(len(style_stats)))
axes[2].set_yticklabels(style_stats.index)
axes[2].set_xlabel('Mean Sale Price ($)')
axes[2].set_title('Average Price by House Style')
axes[2].invert_yaxis()

# Add count labels
for i, (idx, row) in enumerate(style_stats.iterrows()):
    axes[2].text(row['Mean'], i, f"  n={int(row['Count'])}", va='center')

plt.tight_layout()
plt.show()

print("House Style Frequency and Price Analysis:")
print(style_stats.to_string())

**House Style Insights:**
- **Most common:** 1Story (~49%) and 2Story (~30%) dominate the dataset
- **Rare styles:** 2.5Fin (<1%) - very rare, may cause overfitting
- **Price patterns:** 2Story homes command highest average prices, followed by 2.5Unf
- **Single-story paradox:** Despite being most common, 1Story homes have moderate pricing
- **Encoding strategy:** One-hot encoding feasible (8 categories), but consider grouping very rare categories (2.5Fin, 2.5Unf)

#### Detailed Analysis: MS Zoning (Zoning Classification)

In [ ]:
# MS Zoning analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distribution
zoning_counts = df['MS Zoning'].value_counts()
axes[0].bar(range(len(zoning_counts)), zoning_counts.values, color='teal')
axes[0].set_xticks(range(len(zoning_counts)))
axes[0].set_xticklabels(zoning_counts.index, rotation=45)
axes[0].set_ylabel('Count')
axes[0].set_title('MS Zoning Distribution')
axes[0].grid(axis='y', alpha=0.3)

# Price by zoning
df.boxplot(column='SalePrice', by='MS Zoning', ax=axes[1])
axes[1].set_xlabel('Zoning Type')
axes[1].set_ylabel('Sale Price')
axes[1].set_title('Sale Price by Zoning Type')
plt.sca(axes[1])
plt.xticks(rotation=45)

# Mean price comparison
zoning_price = df.groupby('MS Zoning')['SalePrice'].agg(['mean', 'count']).sort_values('mean', ascending=False)
axes[2].barh(range(len(zoning_price)), zoning_price['mean'].values, color='coral')
axes[2].set_yticks(range(len(zoning_price)))
axes[2].set_yticklabels(zoning_price.index)
axes[2].set_xlabel('Mean Sale Price')
axes[2].set_title('Average Price by Zoning')
axes[2].invert_yaxis()

# Add count labels
for i, (idx, row) in enumerate(zoning_price.iterrows()):
    axes[2].text(row['mean'], i, f"  n={int(row['count'])}", va='center')

plt.tight_layout()
plt.show()

print("MS Zoning Statistics:")
print(df.groupby('MS Zoning')['SalePrice'].describe().round(2))

**MS Zoning Insights:**
- **Dominant zones:** RL (Residential Low Density) and RM (Residential Medium Density)
- **Price hierarchy:** Floating Village Residential (FV) commands highest prices, followed by RL
- **Commercial zones:** Very few observations - may need grouping or removal
- **Encoding strategy:** One-hot encoding suitable (only 5-8 categories)

### Categorical Variables Summary: Key Takeaways

**🎯 HIGH PRIORITY ENCODING DECISIONS:**

**1. GROUP RARE CATEGORIES (Critical for model stability):**
- **Roof Style:** Combine Flat, Gambrel, Mansard, Shed → "Other"
- **Foundation:** Combine Stone, Wood, Slab → "Other"  
- **House Style:** Combine 2.5Fin, 2.5Unf → "Other_MultiLevel"

**2. HIGH CARDINALITY FEATURES (Need special encoding):**
- **Neighborhood (28 unique):** Use target encoding or frequency encoding
- **Exterior 1st/2nd (15+ unique):** Target encoding recommended
- Any feature >10 categories: Avoid one-hot encoding

**3. MISSING VALUES AS INFORMATION:**
- **Garage features:** Missing = "No Garage" (informative)
- **Pool QC:** Missing = "No Pool" (informative)
- **Fireplace Qu:** Missing = "No Fireplace" (informative)
- Create binary indicators: `Has_Garage`, `Has_Pool`, `Has_Fireplace`

**4. BALANCED vs IMBALANCED:**
- **Severely imbalanced:** Bldg Type (83% single class), Roof Style (86% Gable)
- **Well balanced:** MS Zoning, House Style (relatively distributed)
- **Implication:** Use stratified sampling during train/test split

**5. RECOMMENDED ENCODING STRATEGIES:**

| Cardinality | Examples | Recommended Encoding |
|------------|----------|---------------------|
| **Low (2-4)** | Street, Utilities, Central Air | One-Hot Encoding |
| **Medium (5-10)** | MS Zoning, Bldg Type, House Style | One-Hot (after grouping rare) |
| **High (>10)** | Neighborhood, Exterior 1st/2nd | Target/Frequency Encoding |

**6. FEATURES TO ENGINEER:**
- `Has_Garage` (binary): Garage presence indicator
- `Has_Pool` (binary): Pool presence indicator  
- `Rare_Neighborhood` (binary): Is neighborhood <5% frequency?
- `Foundation_Quality` (ordinal): PConc=3, BrkTil=2, CBlock=1, Other=0

**🚨 PREPROCESSING CHECKLIST:**
- [ ] Group rare categories (<5% frequency) in 6-8 features
- [ ] Decide encoding strategy per cardinality level
- [ ] Create missing value indicators for informative missingness
- [ ] Document rare category groupings for production pipeline
- [ ] Plan for unseen categories in production (default handling)

## 4. Univariate Analysis

### Target Variable: SalePrice (Numerical)

In [ ]:
# Univariate analysis for SalePrice
fig, axes = plt.subplots(1, 1, figsize=(14, 5))

# Histogram
axes.hist(df['SalePrice'], bins=50, edgecolor='black', color='skyblue')
axes.set_xlabel('Sale Price')
axes.set_ylabel('Frequency')
axes.set_title('Distribution of SalePrice')
axes.axvline(df['SalePrice'].mean(), color='red', linestyle='--', label=f"Mean: ${df['SalePrice'].mean():,.0f}")
axes.axvline(df['SalePrice'].median(), color='green', linestyle='--', label=f"Median: ${df['SalePrice'].median():,.0f}")
axes.legend()

plt.tight_layout()
plt.show()

print(f"SalePrice Statistics:")
print(f"Mean: ${df['SalePrice'].mean():,.2f}")
print(f"Median: ${df['SalePrice'].median():,.2f}")
print(f"Std Dev: ${df['SalePrice'].std():,.2f}")
print(f"Min: ${df['SalePrice'].min():,.2f}")
print(f"Max: ${df['SalePrice'].max():,.2f}")
print(f"Skewness: {df['SalePrice'].skew():.2f}")

### Categorical Feature: Neighborhood

In [ ]:
# Univariate analysis for Neighborhood
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Count plot
neighborhood_counts = df['Neighborhood'].value_counts()
axes[0].bar(range(len(neighborhood_counts)), neighborhood_counts.values)
axes[0].set_xticks(range(len(neighborhood_counts)))
axes[0].set_xticklabels(neighborhood_counts.index, rotation=45, ha='right')
axes[0].set_xlabel('Neighborhood')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Neighborhoods')

# Pie chart
top_neighborhoods = df['Neighborhood'].value_counts().head(10)
axes[1].pie(top_neighborhoods.values, labels=top_neighborhoods.index, autopct='%1.1f%%')
axes[1].set_title('Top 10 Neighborhoods by Count')

plt.tight_layout()
plt.show()

print(f"Total unique neighborhoods: {df['Neighborhood'].nunique()}")
print(f"\nTop 10 Neighborhoods:")
print(df['Neighborhood'].value_counts().head(10))

### Univariate Insights

**SalePrice Distribution:**
- Positively skewed distribution (right tail)
- Most prices between $100k-$250k
- Peak frequency around $200k
==> May benefit from log transformation for modeling

**Neighborhood Distribution:**
- 28 unique neighborhoods
- `NAmes` (North Ames) is the most common
- Followed by `CollgCr` (College Creek) and `OldTown`
- Some neighborhoods have very few observations
==> Consider using frequency encoder

## 5. Bivariate Analysis

### Numerical vs Numerical: Gr Liv Area vs SalePrice

In [ ]:
# Scatter plot with regression line
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(df['Gr Liv Area'], df['SalePrice'], alpha=0.5)
axes[0].set_xlabel('Above Ground Living Area (sq ft)')
axes[0].set_ylabel('Sale Price')
axes[0].set_title('Gr Liv Area vs SalePrice')

# Add regression line
z = np.polyfit(df['Gr Liv Area'], df['SalePrice'], 1)
p = np.poly1d(z)
axes[0].plot(df['Gr Liv Area'], p(df['Gr Liv Area']), "r--", linewidth=2, label='Trend')
axes[0].legend()

# Hexbin plot
hb = axes[1].hexbin(df['Gr Liv Area'], df['SalePrice'], gridsize=30, cmap='YlOrRd')
axes[1].set_xlabel('Above Ground Living Area (sq ft)')
axes[1].set_ylabel('Sale Price')
axes[1].set_title('Density: Gr Liv Area vs SalePrice')
plt.colorbar(hb, ax=axes[1])

plt.tight_layout()
plt.show()

correlation = df['Gr Liv Area'].corr(df['SalePrice'])
print(f"Correlation between Gr Liv Area and SalePrice: {correlation:.4f}")

### Categorical vs Numerical: Overall Qual vs SalePrice

In [ ]:
# Box plot by quality
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='SalePrice', by='Overall Qual', ax=axes[0])
axes[0].set_xlabel('Overall Quality')
axes[0].set_ylabel('Sale Price')
axes[0].set_title('SalePrice by Overall Quality')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Violin plot
sns.violinplot(data=df, x='Overall Qual', y='SalePrice', ax=axes[1])
axes[1].set_xlabel('Overall Quality')
axes[1].set_ylabel('Sale Price')
axes[1].set_title('Distribution of SalePrice by Overall Quality')

plt.tight_layout()
plt.show()

# Mean sale price by quality
print("Mean Sale Price by Overall Quality:")
print(df.groupby('Overall Qual')['SalePrice'].agg(['mean', 'median', 'count']).round(2))

correlation = df['Overall Qual'].corr(df['SalePrice'])
print(f"\nCorrelation between Overall Qual and SalePrice: {correlation:.4f}")

### Bivariate Insights

**Gr Liv Area vs SalePrice:**
- Strong positive correlation (~0.71)
- Linear relationship: Larger homes sell for more
- Some outliers present (large homes with lower prices)
==> Consider a good predictor for the model

**Overall Qual vs SalePrice:**
- Very strong positive correlation (~0.80)
- Clear ordinal relationship across quality levels
- Higher quality consistently associated with higher prices
==? Consider a strong predictors available

## 6. Multivariate Analysis

In [ ]:
# Select important numerical features for correlation analysis
important_features = ['SalePrice', 'Gr Liv Area', 'Overall Qual', 'Total Bsmt SF', 'Year Built', '1st Flr SF']
selected_data = df[important_features].copy()

# Calculate correlation matrix
correlation_matrix = selected_data.corr()

# Display correlation with SalePrice
print("Correlation with SalePrice:")
print(correlation_matrix['SalePrice'].sort_values(ascending=False))
print("\nCorrelation Matrix:")
print(correlation_matrix.round(3))

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f', square=True)
plt.title('Correlation Heatmap of Important Features')
plt.tight_layout()
plt.show()

In [ ]:
# Pair plot for multivariate relationships
from pandas.plotting import scatter_matrix

scatter_matrix(selected_data, figsize=(12, 10), alpha=0.6, diagonal='hist')
plt.tight_layout()
plt.show()

In [ ]:
selected_data[selected_data['Total Bsmt SF'].isnull()]

In [ ]:
# Check for multicollinearity using VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Prepare data (remove SalePrice, use only features)

X = selected_data.drop('SalePrice', axis=1).fillna(selected_data.median())

# Calculate VIF
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF):")
print(vif_data.sort_values('VIF', ascending=False))
print("\nVIF Interpretation:")
print("VIF < 5: Low multicollinearity (generally acceptable)")
print("VIF 5-10: Moderate multicollinearity (may need attention)")
print("VIF > 10: High multicollinearity (should be addressed)")

### Multivariate Insights

**Correlation with SalePrice (sorted):**
1. `Overall Qual`: 0.80 - Strongest predictor
2. `Gr Liv Area`: 0.71 - Strong predictor
3. `Total Bsmt SF`: 0.63 - Moderate predictor
4. `1st Flr SF`: 0.61 - Moderate predictor
5. `Year Built`: 0.56 - Weak-moderate predictor

**Key Findings:**
- Overall Quality and Living Area are the dominant price drivers
- Strong linear relationships support regression modeling
- Multicollinearity present wiht likely reasons:
    - '1st Flr SF' + 'Total Bsmt SF' → Both measure floor space, redundant information
    - 'Gr Liv Area' ⊃ '1st Flr SF' → Living area includes first floor, creating mathematical dependency
    - 'Overall Qual' ↔ Size → Quality tends to increase with size (confounded relationship)
    - 'Year Built' ↔ Quality → Newer homes rated higher quality (temporal confounding)

**For Modeling:**
- Include top correlated features: Overall Qual, Gr Liv Area, Total Bsmt SF. Drop '1st Flr SF' as it is redundant
- Use multicollinearity-proof models such as regularization regression models suhc as L1, L2,and Elastic Net
- Consider feature selection or PCA to reduce redundancy
- Log transformation of SalePrice may improve model performance